# Задание 1

<br>
Есть таблица с действиями юзеров A/B-теста (<i>task_1_events.csv</i>):

- $events$ - установки приложения
    - $user\_id$ - id юзера,
    - $ab\_group$ - группа A/B-теста,
    - $ts$ - время совершения действия,
    - $pdate$ - дата совершения действия.

Пользовательская сессия определяется по следующим правилам:
<br>
1. Новая сессия начинается после 30 минут бездействия.
<br>
2. Сессия прерывается при переходе между двумя датами.

<b>Постройте таблицу с сессиями юзеров в формате<b>:
<br>
- $user\_id$ - id юзера
- $ab\_group$ - группа A/B-теста,
- $start\_ts$ - время старта сессии,
- $end\_ts$ - время окончания сессии,
- $pdate$ - дата сессии.

# Задание 2

Был проведен A/B-тест.
<br>
В качестве данных используйте таблицу, построенную в предыдущем задании. Первая сессия юзера считается моментом попадания в A/B-тест.
<br><br>
Ключевая метрика эксперимента - конверсия во вторую сессию.
<br>
Сделайте вывод о том, какая группа выиграла в A/B-тесте. Ответ обоснуйте.
<br>

Импорт библиотек

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import datetime as dt
from pprint import pprint


Считаем файл с данными в фатафрейм 

In [2]:
df = pd.read_csv('task_1_events.csv')
df.head()

,user_id,ab_group,ts,pdate
0,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:33.553327,2023-05-02
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:32:23.215284,2023-05-02
2,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 13:33:01.050395,2023-05-02
3,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 13:36:45.210274,2023-05-02
4,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 13:36:54.251438,2023-05-02


In [3]:
df['ts'] = pd.to_datetime(df['ts']).round('1s') # пребразуем тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44567 entries, 0 to 44566
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   44567 non-null  object        
 1   ab_group  44567 non-null  object        
 2   ts        44567 non-null  datetime64[ns]
 3   pdate     44567 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.4+ MB


Посмотрим какое у нас количество пользователей участвует в тесте 

In [4]:
len(df['user_id'].unique())

5000

Отсортируем данные по пользователям и времени события 

In [5]:
df = df.sort_values(by=['user_id', 'ts']).reset_index(drop=True)
df.head(10)

,user_id,ab_group,ts,pdate
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:46,2023-05-01
1,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23,2023-05-01
2,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24,2023-05-01
3,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:27,2023-05-01
4,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28,2023-05-01
5,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:55,2023-05-01
6,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:20,2023-05-01
7,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01
8,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01
9,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:23,2023-05-01


In [6]:
# Вычислим разницу во времени между  событиями каждого юзера
df['time_diff'] = df.groupby('user_id')['ts'].diff()
df.head(10)

,user_id,ab_group,ts,pdate,time_diff
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:46,2023-05-01,NaT
1,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23,2023-05-01,0 days 00:00:37
2,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24,2023-05-01,0 days 00:00:01
3,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:27,2023-05-01,0 days 00:00:03
4,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28,2023-05-01,0 days 00:00:01
5,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:55,2023-05-01,0 days 00:00:27
6,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:20,2023-05-01,0 days 00:04:25
7,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01,0 days 00:00:01
8,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01,0 days 00:00:00
9,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:23,2023-05-01,0 days 00:00:02


Добавим новый столбец, индикатор для начала нового дня. Он нужен нам т к в задании есть ремарка что сессия прерывается между датами 

In [7]:
for i, row in df.iterrows(): #
    if i == 0:    # если первая строка 
        continue # пропускаем первую  итерацию
    df.loc[i,'new_day'] = (row['ts'].date() != df.loc[i-1, 'ts'].date())

Посмторим что у нас вышло 

In [8]:
df.head(10)

,user_id,ab_group,ts,pdate,time_diff,new_day
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:46,2023-05-01,NaT,NaN
1,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23,2023-05-01,0 days 00:00:37,False
2,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24,2023-05-01,0 days 00:00:01,False
3,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:27,2023-05-01,0 days 00:00:03,False
4,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28,2023-05-01,0 days 00:00:01,False
5,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:55,2023-05-01,0 days 00:00:27,False
6,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:20,2023-05-01,0 days 00:04:25,False
7,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01,0 days 00:00:01,False
8,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01,0 days 00:00:00,False
9,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:23,2023-05-01,0 days 00:00:02,False


In [9]:
# Определяем новую сессию, если прошло более 30 минут с момента предыдущего события или если поле time_diff пустое (первое действие пользователя ), или если изменена дата
df['new_session'] = ((df['time_diff'] > pd.Timedelta(minutes=30)) | (df['time_diff'].isnull()) | df['new_day'] ) 
df.head(20)

,user_id,ab_group,ts,pdate,time_diff,new_day,new_session
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:46,2023-05-01,NaT,NaN,True
1,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23,2023-05-01,0 days 00:00:37,False,False
2,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24,2023-05-01,0 days 00:00:01,False,False
3,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:27,2023-05-01,0 days 00:00:03,False,False
4,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28,2023-05-01,0 days 00:00:01,False,False
5,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:55,2023-05-01,0 days 00:00:27,False,False
6,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:20,2023-05-01,0 days 00:04:25,False,False
7,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01,0 days 00:00:01,False,False
8,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21,2023-05-01,0 days 00:00:00,False,False
9,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:23,2023-05-01,0 days 00:00:02,False,False


In [10]:
# Находим индексы, где начинаются новые сессии
session_start_indices = df[df['new_session']].index
print(session_start_indices)

Int64Index([    0,    18,    20,    23,    25,    31,    33,    34,    35,
               36,
            ...
            44497, 44500, 44503, 44520, 44525, 44531, 44537, 44542, 44551,
            44558],
           dtype='int64', length=5929)


In [11]:
# Определяем время начала и окончания каждой сессии
sessions = []
for i in range(len(session_start_indices)):
    start_idx = session_start_indices[i]
    end_idx = session_start_indices[i+1] if i+1 < len(session_start_indices) else len(df)
    session = df.iloc[start_idx:end_idx]
    sessions.append({
        'user_id': session['user_id'].iloc[0],
        'ab_group': session['ab_group'].iloc[0],
        'start_ts': session['ts'].iloc[0],
        'end_ts': session['ts'].iloc[-1],
        'pdate': session['pdate'].iloc[0]
    })

Проеобразуем полученный список со словарями в датафрейм

In [13]:
sessions_df = pd.DataFrame(sessions)
sessions_df.head()

,user_id,ab_group,start_ts,end_ts,pdate
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:46,2023-05-01 08:57:37,2023-05-01
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:34,2023-05-02 12:32:23,2023-05-02
2,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 13:33:01,2023-05-02 13:36:54,2023-05-02
3,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 14:37:39,2023-05-02 14:37:58,2023-05-02
4,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-01 18:58:45,2023-05-01 19:04:19,2023-05-01


In [15]:
sessions_df.shape

(5929, 5)

    Новая таблица содержит 5929 строк 

In [19]:
sessions_df.groupby(['ab_group', 'user_id']).size().reset_index().groupby('ab_group').size()# проверяем количество пользователей в каждой группе

ab_group
A    2500
B    2500
dtype: int64

In [17]:
sessions_df.groupby(['user_id', 'ab_group']).agg('size') # сгруппируем данные по пользователям и по группам, посмотрим кол-во сессий у каждого пользователя  

user_id                   ab_group
2JM4L+sRXYIWAFSC8XaTPg==  B           1
2OgK/ukRgvkWAOaa8XaTPg==  A           3
3E69ZO0RJrEWACi28XaTPg==  A           5
3GsYmewRRswWABqG8XaTPg==  B           1
4BNABuYRf+8WAKmUYvZgPg==  A           1
                                     ..
ruaymOsRv0UWAPiO6VfbPg==  A           2
wtgmjOgR/qoWAB2+8XaTPg==  B           1
xENZPuwRxj0WAPuQ6VfbPg==  A           1
xG+5l+wRnG0WAPOg6VfbPg==  B           1
xppk/u0RbGIWAKC+6VfbPg==  A           7
Length: 5000, dtype: int64

    Таблицу sessions_df группируем по user_id и ab_group и подсчитываем количество сессий по каждому пользователю , 
    далее оставляем только тех у кого кол-во сессий более 1й, затем снова группируем по ab_group и считаем кол-во поль-лей

In [18]:
sessions_df.groupby(['user_id', 'ab_group']).agg('size').loc[lambda x: x>1].groupby('ab_group').agg('size')

ab_group
A    319
B    353
dtype: int64

    Таким образом мы видим кол-во пользователей по каждой группе совершивших несколько сессий 
    A - 319
    B - 353
    Итог: группа В совершила большее кол-во сессий (более 1), а значит победила в АВ-тесте 